### Welcome to the exercise about observational studies! This exercise will be hands on, and you will be able to practise the skills you developed so far!

## Propensity score matching

In this exercise, you will apply [propensity score matching](http://www.stewartschultz.com/statistics/books/Design%20of%20observational%20studies.pdf), which we discussed in lecture 6 ("Observational studies"), in order to draw conclusions from an observational study.

We will work with a by-now classic dataset from Robert LaLonde's study "[Evaluating the Econometric Evaluations of Training Programs](http://people.hbs.edu/nashraf/LaLonde_1986.pdf)" (1986).
The study investigated the effect of a job training program ("National Supported Work Demonstration") on the real earnings of an individual, a couple of years after completion of the program.
Your task is to determine the effectiveness of the "treatment" represented by the job training program.

#### Dataset description

- `treat`: 1 if the subject participated in the job training program, 0 otherwise
- `age`: the subject's age
- `educ`: years of education
- `race`: categorical variable with three possible values: Black, Hispanic, or White
- `married`: 1 if the subject was married at the time of the training program, 0 otherwise
- `nodegree`: 1 if the subject has earned no school degree, 0 otherwise
- `re74`: real earnings in 1974 (pre-treatment)
- `re75`: real earnings in 1975 (pre-treatment)
- `re78`: real earnings in 1978 (outcome)

If you want to deepen your knowledge on propensity scores and observational studies, we highly recommend Rosenbaum's excellent book on the ["Design of Observational Studies"](http://www.stewartschultz.com/statistics/books/Design%20of%20observational%20studies.pdf). Even just reading the first chapter (18 pages) will help you a lot.

In [ ]:
import pandas as pd
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx
%matplotlib inline

data_folder = './data/'
df = pd.read_csv(data_folder + 'lalonde.csv')

In [ ]:
df.sample(10)

## 1. A naive analysis

Compare the distribution of the outcome variable (`re78`) between the two groups, using plots and numbers.
To summarize and compare the distributions, you may use the techniques we discussed in lecture 4 ("Descibing data") and 3 ("Visualizing data").

What might a naive "researcher" conclude from this superficial analysis?

In [ ]:
df_training = df[df['treat'] == 1]['re78']
df_no_training = df[df['treat'] == 0]['re78']

print(df_training.describe())
print(df_no_training.describe())their 

fig, ax = plt.subplots(2,figsize= (8,6), sharey = True, sharex = True)

ax[0].hist(df_training, bins = 100)
ax[0].set_title('1978 salaries for trained subjects')
ax[1].hist(df_no_training, bins = 100)
ax[1].set_title('1978 salaries for untrained subjects')


fig.text(0.4,0, "Real earnings in 1978")
fig.text(0,0.6, "Number of subjects", rotation = 90)

plt.show()

In [ ]:
plt.boxplot([df_training, df_no_training], tick_labels=['1978 salaries for trained subjects', '1978 salaries for untrained subjects'])
plt.show()

From the naive observation it can be concluded that on average there is no significant difference between the salaries of the different subjects, but there are more trained subject outliers with higher salaries that the median.

## 2. A closer look at the data

You're not naive, of course (and even if you are, you've learned certain things in ADA), so you aren't content with a superficial analysis such as the above.
You're aware of the dangers of observational studies, so you take a closer look at the data before jumping to conclusions.

For each feature in the dataset, compare its distribution in the treated group with its distribution in the control group, using plots and numbers.
As above, you may use the techniques we discussed in class for summarizing and comparing the distributions.

What do you observe?
Describe what your observations mean for the conclusions drawn by the naive "researcher" from his superficial analysis.

In [ ]:
def display_boxplot(column):
    df_training = df[df['treat'] == 1][column]
    df_no_training = df[df['treat'] == 0][column]

    print(df_training.describe())
    print(df_no_training.describe())

    plt.boxplot([df_training, df_no_training], tick_labels=[column + ' for trained subjects', column + ' for untrained subjects'])
    plt.show()

def display_scatter(column):
    df_training = df[df['treat'] == 1][column]
    df_no_training = df[df['treat'] == 0][column]

    print(df_training.describe())
    print(df_no_training.describe())

    ct = pd.crosstab(df[column], df['treat'])
    sns.heatmap(ct)
    plt.show()

display_boxplot('age')
display_boxplot('educ')
display_boxplot('re74')
display_boxplot('re75')
display_scatter('black')
display_scatter('hispan')
display_scatter('married')
display_scatter('nodegree')

It is visible that there is a correlation between all pairs of columns given that there are differences in the values for each column in the dataset depending on the rows being in the control or test group.

## 3. A propensity score model

Use logistic regression to estimate propensity scores for all points in the dataset.
You may use `statsmodels` to fit the logistic regression model and apply it to each data point to obtain propensity scores.

Recall that the propensity score of a data point represents its probability of receiving the treatment, based on its pre-treatment features (in this case, age, education, pre-treatment income, etc.).
To brush up on propensity scores, you may read chapter 3.3 of the above-cited book by Rosenbaum or [this article](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3144483/pdf/hmbr46-399.pdf).

Note: you do not need a train/test split here. Train and apply the model on the entire dataset. If you're wondering why this is the right thing to do in this situation, recall that the propensity score model is not used in order to make predictions about unseen data. Its sole purpose is to balance the dataset across treatment groups.
(See p. 74 of Rosenbaum's book for an explanation why slight overfitting is even good for propensity scores.
If you want even more information, read [this article](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3144483/pdf/hmbr46-399.pdf).)

In [ ]:
import statsmodels.api as sm

y = df['treat']

x = df[['age', 'educ', 'black', 'hispan', 'married', 'nodegree', 're74', 're75', 're78']]

X = sm.add_constant(x)

ps_model = sm.Logit(y, X).fit()

print(ps_model.summary())

In [ ]:
df['propensity_score'] = ps_model.predict(X)

print(df[['treat', 'propensity_score']].head())

## 4. Balancing the dataset via matching

Use the propensity scores to match each data point from the treated group with exactly one data point from the control group, while ensuring that each data point from the control group is matched with at most one data point from the treated group.
(Hint: you may explore the `networkx` package in Python for predefined matching functions.)

Your matching should maximize the similarity between matched subjects, as captured by their propensity scores.
In other words, the sum (over all matched pairs) of absolute propensity-score differences between the two matched subjects should be minimized.

This is how networkx library can help you do this. Each possible pair of (treated_person, control_person) is characterized by a similarity. This is how we can initialize a graph, and add an edge for one possible pair. We then need to add an edge for each possible pair.
    - G = nx.Graph()
    - G.add_weighted_edges_from([(control_person, treated_person, similarity)])
Optimal matching is then found with:
    - matching = nx.max_weight_matching(G)

After matching, you have as many treated as you have control subjects.
Compare the outcomes (`re78`) between the two groups (treated and control).

Also, compare again the feature-value distributions between the two groups, as you've done in part 2 above, but now only for the matched subjects.
What do you observe?
Are you closer to being able to draw valid conclusions now than you were before?

In [ ]:
treated = df[df['treat'] == 1].copy()
control = df[df['treat'] == 0].copy()

G = nx.Graph()

for i, row in treated.iterrows():
    G.add_node(f"t_{i}", bipartite=0)

for j, row in control.iterrows():
    G.add_node(f"c_{j}", bipartite=1)

for i, t_row in treated.iterrows():
    for j, c_row in control.iterrows():
        weight = abs(t_row['propensity_score'] - c_row['propensity_score'])
        G.add_edge(f"t_{i}", f"c_{j}", weight=weight)

In [ ]:
from networkx.algorithms import matching

matched_edges = matching.min_weight_matching(G, weight='weight')

print(f"Number of matched pairs: {len(matched_edges)}")

In [ ]:
matched_pairs = []

for t_node, c_node in matched_edges:
    if t_node.startswith('c'):
        t_node, c_node = c_node, t_node

    t_idx = int(t_node.split('_')[1])
    c_idx = int(c_node.split('_')[1])
    matched_pairs.append((t_idx, c_idx))

matched_df = pd.DataFrame(matched_pairs, columns=['treated_idx', 'control_idx'])

matched_df['treated_ps'] = matched_df['treated_idx'].apply(lambda i: df.loc[i, 'propensity_score'])
matched_df['control_ps'] = matched_df['control_idx'].apply(lambda i: df.loc[i, 'propensity_score'])
matched_df['ps_diff'] = abs(matched_df['treated_ps'] - matched_df['control_ps'])

print(matched_df.head())
print('Average propensity score difference:', matched_df['ps_diff'].mean())

## 5. Balancing the groups further

Based on your comparison of feature-value distributions from part 4, are you fully satisfied with your matching?
Would you say your dataset is sufficiently balanced?
If not, in what ways could the "balanced" dataset you have obtained still not allow you to draw valid conclusions?

Improve your matching by explicitly making sure that you match only subjects that have the same value for the problematic feature.
Argue with numbers and plots that the two groups (treated and control) are now better balanced than after part 4.


In [ ]:
# 1. Select numeric columns only
numeric_cols = df.select_dtypes(include=[np.number]).columns

# 2. Compute means for treated and control groups
treated_mean = treated_matched[numeric_cols].mean()
control_mean = control_matched[numeric_cols].mean()

# 3. Compute balance table
balance = pd.DataFrame({
    'treated_mean': treated_mean,
    'control_mean': control_mean
})

# 4. Compute standardized mean differences
stds = df[numeric_cols].std()
balance['std_diff'] = (balance['treated_mean'] - balance['control_mean']) / stds

# 5. Display
print(balance.sort_values('std_diff', key=abs))

# 6. Plot standardized differences
plt.figure(figsize=(8,5))
balance['std_diff'].abs().sort_values().plot(kind='barh')
plt.axvline(0.1, color='r', linestyle='--', label='Threshold (0.1)')
plt.xlabel('|Standardized Mean Difference|')
plt.title('Covariate Balance After Matching')
plt.legend()
plt.show()

In [ ]:
G = nx.Graph()

for i in treated.index:
    for j in control.index:
        # Only connect nodes if 'black' value is the same
        if df.loc[i, 'black'] == df.loc[j, 'black']:
            weight = abs(df.loc[i, 'propensity_score'] - df.loc[j, 'propensity_score'])
            G.add_edge(f"t_{i}", f"c_{j}", weight=weight)

matched_edges = nx.algorithms.matching.min_weight_matching(G, weight='weight')

matched_records = []
for a, b in matched_edges:
    # Identify which node is treated and which is control
    if a.startswith('t_'):
        treated = int(a.replace('t_', ''))
        control = int(b.replace('c_', ''))
    else:
        treated = int(b.replace('t_', ''))
        control = int(a.replace('c_', ''))

    matched_records.append({
        'treated_idx': treated,
        'control_idx': control,
        'treated_ps': df.loc[treated, 'propensity_score'],
        'control_ps': df.loc[control, 'propensity_score'],
        'ps_diff': abs(
            df.loc[treated, 'propensity_score'] -
            df.loc[control, 'propensity_score']
        )
    })

matched_df2 = pd.DataFrame(matched_records)
print(matched_df2.head())

In [ ]:
treated_matched2 = df.loc[matched_df2['treated_idx']]
control_matched2 = df.loc[matched_df2['control_idx']]

# Compare distributions again
for col in ['black', 'educ', 'married', 're74']:
    sns.kdeplot(treated_matched2[col], label='Treated', fill=True)
    sns.kdeplot(control_matched2[col], label='Control', fill=True)
    plt.title(f"{col}: After exact matching on 'black'")
    plt.legend()
    plt.show()

## 6. A less naive analysis

Compare the outcomes (`re78`) between treated and control subjects, as you've done in part 1, but now only for the matched dataset you've obtained from part 5.
What do you conclude about the effectiveness of the job training program?

In [ ]:
treated_matched2 = df.loc[matched_df2['treated_idx']]
control_matched2 = df.loc[matched_df2['control_idx']]

# Compare mean re78 (earnings after training)
treated_mean = treated_matched2['re78'].mean()
control_mean = control_matched2['re78'].mean()

effect = treated_mean - control_mean

print(f"Mean re78 (treated): {treated_mean:.2f}")
print(f"Mean re78 (control): {control_mean:.2f}")
print(f"Estimated treatment effect (ATT): {effect:.2f}")


In [ ]:
from scipy import stats

t_stat, p_value = stats.ttest_ind(treated_matched2['re78'], control_matched2['re78'])
print(f"T-statistic: {t_stat:.3f}, p-value: {p_value:.3f}")


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

sns.kdeplot(treated_matched2['re78'], label='Treated', fill=True)
sns.kdeplot(control_matched2['re78'], label='Control', fill=True)
plt.title('Distribution of re78 (Post-treatment earnings)')
plt.legend()
plt.show()
